In [9]:
import torch
import torch.nn as nn
import tiktoken


In [10]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers (# of transformer blocks)
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [11]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):    # cfg refers to the gpt config dictionary above
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]) # makes/unpacks a list of transformer blocks, stuck together as modules
        
        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits   # tensor of scores (like probabilities: logits = log of the odds of something happening)

# not doing anything yet
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

In [12]:
# just an example of unpacking or * function
primes = [2, 3, 5, 7, 11]
print(primes)
print(*primes)

[2, 3, 5, 7, 11]
2 3 5 7 11


In [14]:
tokenizer = tiktoken.get_encoding("gpt2")

In [15]:
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)
# one day will output probabilities of the next word being that word in the vocab (highest score is the most probably next word in the vocab)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.1517, -0.5300,  1.4469,  ..., -0.9767, -0.2406, -1.0669],
         [-2.9517, -0.5859, -0.4100,  ..., -0.0414, -0.9889, -1.0376],
         [-1.3955,  0.3102,  0.5237,  ..., -0.2854,  0.5742,  0.5786],
         [ 1.6111, -0.1006, -0.2744,  ...,  0.1705,  1.3653,  0.2857]],

        [[-0.6413, -0.2182,  1.2814,  ..., -0.3877, -0.5411, -0.3671],
         [-2.4538, -1.4791, -0.2876,  ..., -0.1777, -0.2182, -0.9644],
         [-0.4809,  0.7123, -0.2452,  ...,  1.0891,  0.3373,  0.1368],
         [ 0.6394,  0.0962,  0.4398,  ..., -0.2997,  0.6460, -1.4245]]],
       grad_fn=<UnsafeViewBackward0>)
